# API food : recommender

In [2]:
#import getpass
import pandas as pd
import requests
from IPython.display import display

### Recommender Proposal
1. input : user types 1 or several ingredients
2. output: app gives 10 proposal recipes
3. input: user choses 1 recipe out of the 10 (e.g. "risotto chicken")
4. output: model proposes a another similar recipe ("if you liked 'chicken risotto' I recommend you try 'paprika pasta')
    MODEL: recipe clustering based on nutrition , number of ingredient, number of steps, taste,  language processing ~

# Get clustered data

In [3]:
data=pd.read_csv('recipes_w_clusters.csv')

# Recommender

In [4]:
with open("spoonacular-api-key.txt", "r") as file:
        api_key = file.readline().split(':')[1].strip()
api_key

'd3f78262728d464a98045ef32eff5060'

In [8]:

# Read Spoonacular API Key from file
with open("spoonacular-api-key.txt", "r") as file:
        api_key = file.readline().split(':')[1].strip()
        
# Define the endpoint you want to access (for example, ingredient search)
endpoint = 'https://api.spoonacular.com/recipes/findByIngredients'

# Get the list of ingredients from the user
user_input = input("Enter the ingredients in your fridge (comma-separated): ")
fridge_contents = user_input.split(',')

# Define parameters for the request
params = {
    'ingredients': ','.join(fridge_contents),  # Convert list to comma-separated string
    'apiKey': api_key
}

# GET request to API
response = requests.get(endpoint, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the data from the response (in JSON format)
    recipes = response.json()
    # Convert the recipe data to a DataFrame
    recipe_data = []
    for recipe in recipes:
        recipe_data.append({
            '10 Recipes for you': recipe['title'],
            'You will need those additional ingredients': ', '.join([ingredient['name'] for ingredient in recipe['missedIngredients']])
        })
    recipes_df = pd.DataFrame(recipe_data)
    display(recipes_df)
## Prompt the user to select a preferred recipe
    selected_recipe_title = input("Which of those recipes do you prefer? Enter the recipe title: ")
    
## Search for the selected recipe in the 'data' DataFrame
    selected_recipe = data[data['title'] == selected_recipe_title].head(1)
    #display(selected_recipe)
    if len(selected_recipe) > 0:
        # Display the details of the selected recipe
        print("\nHere are some FYI details for this recipe:")
        for index, row in selected_recipe.iterrows():
            print(f"Your prefered recipe: {row['title']}")
            print(f"Health Score (out of 100): {row['healthScore']}")
            print(f"Price Per Serving: ${row['pricePerServing']}")
            #print(f"Summary: {row['summary']}")
## Suggest another recipe from the model
         # Get the cluster number of the selected recipe
        selected_cluster = selected_recipe.iloc[0]['cluster']
        
        # Filter data for recipes in the same cluster as the selected recipe
        cluster_recipes = data[data['cluster'] == selected_cluster]
    
        # Remove the selected recipe from the cluster if it exists
        cluster_recipes = cluster_recipes[cluster_recipes['title'] != selected_recipe_title]
    
        # Choose a random recipe from the same cluster
        if len(cluster_recipes) > 0:
            random_recommendation = cluster_recipes.sample(1)
            print("\nGiven your fine taste, may I suggest:")
            print(f"a recipe you may like: {random_recommendation['title'].iloc[0]}")
            print(f"Health Score: {random_recommendation['healthScore'].iloc[0]}")
            print(f"Price Per Serving: ${random_recommendation['pricePerServing'].iloc[0]}")
            #print(f"Summary: {random_recommendation['summary'].iloc[0]}")
        else:
            print("Excellent choice.")
    else:
        print("Excellent choice!")


else:
    # Print an error message if the request was not successful
    print('Error:', response.status_code)

Enter the ingredients in your fridge (comma-separated):  cilantro, pepper


,10 Recipes for you,You will need those additional ingredients
0,Balti Chicken,"boned and skinned chicken breast halves, rice,..."
1,Lentil Mango Salad,"lentils, garlic, mango, onion, white wine vinegar"
2,Charred Corn and Pepper Salsa,"ears of corn, limes, onion"
3,Sweet Potato with Bacon and Poached Egg,"bacon, egg, sweet potato"
4,Beef Fajita Marinade,"garlic, ground chipotle pepper, lime juice, li..."
5,Crab Salad Stuffed Pita Pockets,"apple, crabmeat, mayonnaise, wholewheat pita b..."
6,Kickin’ Vegan Quesadillas,"vegan tortillas - 10, vegetarian refried beans..."
7,Mango Nectarine Salsa,"nectarines, mango, lime, jalapeno, avocado"
8,Steamed Clams In Wine and Chorizo,"chorizo cut to, cumin seeds, wine, garlic clov..."
9,Grilled Corn Side Salad,"avocado, ears of corn, honey, jalapeno, juice ..."


Which of those recipes do you prefer? Enter the recipe title:  Charred Corn and Pepper Salsa


Excellent choice!


In [ ]:
#import getpass
#apikey = getpass.getpass() 
#enter password which is 'd3f78262728d464a98045ef32eff5060'

In [48]:
def get_recipe_id(recipe_title):
    endpoint = 'https://api.spoonacular.com/recipes/complexSearch'
    params = {
        'apiKey': api_key,
        'query': recipe_title
    }
    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        results = response.json()
        if results:
            return results['results'][0]['id']  # the first recipe is the one we want
    return None

def get_recipe_instructions(recipe_title):
    recipe_id = get_recipe_id(recipe_title)
    if recipe_id:
        endpoint = f'https://api.spoonacular.com/recipes/{recipe_id}/information'
        params = {
            'apiKey': api_key
        }
        response = requests.get(endpoint, params=params)
        
        if response.status_code == 200:
            recipe_info = response.json()
            return recipe_info.get('instructions', 'Instructions not available.')
            return recipe_info
    return "Mamma mia! I cannot find my cookbook for this one."
    

In [49]:
get_recipe_instructions('Vegetable Dip')

'<ol><li>In a medium bowl combine all ingredients (except raw vegetables for dipping). Cover and refrigerate until ready to serve. Serve as a dip for raw vegetables. Makes approximately 3 cups.</li></ol>'

In [46]:
get_recipe_id('Vegetable Dip')

664547

In [25]:
type(get_recipe_id('Vegetable Dip'))

int

In [ ]:
# TRIAL 1
import numpy as np
import pandas as pd
import streamlit as st
from PIL import Image
# Import file
data=pd.read_csv('recipes_w_clusters.csv')
# Import user modules
#from functions import cleanOperation, cleankSymbol, cleanDuration, preprocess
from IPython.display import display
from getpass import getpass

def print_recipe_details(recipe_title):
    # Ideally, replace this function with your logic to fetch details about the selected recipe
    print("Functionality to print recipe details is not implemented yet.")

def suggest_recipes():
    with open("spoonacular-api-key.txt", "r") as file:
        api_key = file.readline().split(':')[1].strip()
    endpoint = 'https://api.spoonacular.com/recipes/findByIngredients'

    # Get the list of ingredients from the user
    user_input = input("Enter the ingredients in your fridge (comma-separated): ")
    fridge_contents = user_input.split(',')

    # Define parameters for the request
    params = {
        'ingredients': ','.join(fridge_contents),
        'apiKey': api_key
    }

    # GET request to API
    response = requests.get(endpoint, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Extract the data from the response (in JSON format)
        recipes = response.json()
        # Convert the recipe data to a DataFrame
        recipe_data = []
        for recipe in recipes:
            recipe_data.append({
                '10 Recipes for you': recipe['title'],
                'You will need those additional ingredients': ', '.join([ingredient['name'] for ingredient in recipe['missedIngredients']])
            })
        recipes_df = pd.DataFrame(recipe_data)
        display(recipes_df)

        # Prompt the user to select a preferred recipe
        selected_recipe_title = input("Which of those recipes do you prefer? Enter the recipe title: ")

        # Suggest another recipe from the model
        print("\nHere are some FYI details for this recipe:")
        print_recipe_details(selected_recipe_title)

    else:
        # Print an error message if the request was not successful
        print('Error:', response.status_code)



# Example usage:
#api_key = 'd3f78262728d464a98045ef32eff5060'
#suggest_recipes(api_key)


In [ ]:
# TRIAL 2
# Import file
data=pd.read_csv('recipes_w_clusters.csv')
# Import user modules
#from functions import cleanOperation, cleankSymbol, cleanDuration, preprocess
from IPython.display import display

def print_recipe_details(recipe_title):
    # Ideally, replace this function with your logic to fetch details about the selected recipe
        #display(selected_recipe)
    selected_recipe = data[data['title'] == recipe_title].head(1)
    if len(selected_recipe) > 0:
        # Display the details of the selected recipe
        print("\nBelow are some details about this recipe (but who cares about health anyway):")
        for index, row in selected_recipe.iterrows():
            st.write(row['title'])
            st.write(f"Health Score (out of 100): {row['healthScore']}")
            st.write(f"Price Per Serving: ${row['pricePerServing']}")
            
    else: st.write("Mamma mia! I cannot find my cookbook for this one")
    
def suggest_recipes():
    with open("spoonacular-api-key.txt", "r") as file:
        api_key = file.readline().split(':')[1].strip()
    endpoint = 'https://api.spoonacular.com/recipes/findByIngredients'

    # Get the list of ingredients from the user
    user_input = st.text_input("1. Enter the ingredients in your fridge or pantry (comma-separated): ")
    fridge_contents = user_input.split(',') if user_input else []

    # Define parameters for the request
    params = {
        'ingredients': ','.join(fridge_contents),
        'apiKey': api_key
    }

    response = requests.get(endpoint, params=params)

    # if request works
    if response.status_code == 200:
        # Extract the data from the response (in JSON format)
        recipes = response.json()
        # Convert the recipe data to a DataFrame
        recipe_data = []
        for recipe in recipes:
            recipe_data.append({
                'RECIPES! Here is my TOP 10 including your ingredients': recipe['title'],
                'INGREDIENTS: You will need to add those': ', '.join([ingredient['name'] for ingredient in recipe['missedIngredients']])
            })
        recipes_df = pd.DataFrame(recipe_data)
        st.write(recipes_df)

    #### Prompt the user to select a preferred recipe
        selected_recipe_title = st.text_input("2. Which of those recipes do you prefer? Copy-paste your favorite recipe title (case sensitive): ")
        selected_recipe = data[data['title'] == selected_recipe_title].head(1)
        
        print_recipe_details(selected_recipe_title)
        
        if len(selected_recipe) > 0:
            st.write("\nHere are some FYI details for this recipe:")
            print_recipe_details(selected_recipe_title)
        
            
### Suggest another recipe from the model
        
        # Get cluster number of the selected recipe
            selected_cluster = selected_recipe.iloc[0]['cluster']
            # Filter data for recipes in the same cluster as the selected recipe
            cluster_recipes = data[data['cluster'] == selected_cluster]
        
            # Remove the selected recipe from the cluster if it exists
            cluster_recipes = cluster_recipes[cluster_recipes['title'] != selected_recipe_title]
           # Choose a random recipe from the same cluster
            random_recommendation = cluster_recipes.sample(1)
            st.write("\nGiven your fine taste, may I risk a suggestion:", random_recommendation['title'].iloc[0])
            print_recipe_details(random_recommendation['title'].iloc[0])
            
        else:st.write("\nExcellent choice !")
        
    else:
        # Print an error message if the request was not successful
        st.write('Error:', response.status_code)

# Run the app
if __name__ == '__main__':
    suggest_recipes()